<a href="https://colab.research.google.com/github/merliou/term_paper_css/blob/main/stm_try_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas nltk gensim pyLDAvis
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
# 1. Daten laden und vorbereiten
def load_and_preprocess(file_path):
    """Lädt die Daten, entfernt Stoppwörter, stemmt und bereinigt den Text."""
    df = pd.read_csv(r'/content/06_data_raw_no_doubles.csv')

    # Text in Kleinbuchstaben umwandeln
    df['text'] = df['text'].str.lower()

    # Satzzeichen entfernen
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Stoppwörter entfernen
    stop_words = set(stopwords.words('german'))  # oder 'english', falls die Rezensionen auf Englisch sind
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # Stemming
    stemmer = SnowballStemmer('german')  # oder 'english'
    df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

    return df


# 2. Dokument-Term-Matrix erstellen
def create_document_term_matrix(df):
    """Erstellt eine Dokument-Term-Matrix."""
    documents = [doc.split() for doc in df['text']]
    dictionary = Dictionary(documents)

    # Seltende und häufige Wörter filtern (entsprechend der Studie)
    dictionary.filter_extremes(no_below=int(len(documents) * 0.01), no_above=0.2)

    corpus = [dictionary.doc2bow(doc) for doc in documents]
    return corpus, dictionary


# 3. STM mit LDA approximieren und Modell evaluieren
def train_lda_model(corpus, dictionary, num_topics, passes=10, iterations=400):
    """Trainiert das LDA-Modell."""
    lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=passes, iterations=iterations)
    return lda_model


# 4. Themen visualisieren und interpretieren
def visualize_topics(lda_model, corpus, dictionary):
    """Visualisiert die Themen mit pyLDAvis."""
    vis = gensimvis.prepare(lda_model, corpus, dictionary)
    pyLDAvis.display(vis)


# 5. Durchführung (Beispiel)
if __name__ == '__main__':
    file_path = 'your_data.csv'  # Anpassen!
    df = load_and_preprocess(file_path)
    corpus, dictionary = create_document_term_matrix(df)

    # Modellauswahl (basierend auf Kohärenz und Exklusivität - hier nur ein Beispiel)
    num_topics = 50  # Hier die Anzahl der Themen wählen, die am besten passen
    lda_model = train_lda_model(corpus, dictionary, num_topics)

    visualize_topics(lda_model, corpus, dictionary)

    # Themen interpretieren (die wichtigsten Wörter und Beispiele anzeigen lassen)
    for topic_id in range(num_topics):
        top_words = lda_model.show_topic(topic_id, topn=10)  # Top 10 Wörter pro Thema
        print(f"Topic {topic_id}: {', '.join([word for word, prob in top_words])}")